In [ ]:
# %%
import os
import math
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import requests
from dotenv import load_dotenv

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# GPU?
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)


In [ ]:
from dotenv import load_dotenv
import os

# Load the .env file from the project root
load_dotenv(dotenv_path="AI/.env")  # adjust path if notebook is in /notebooks

OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
print("API key loaded:", OPENWEATHER_API_KEY)


In [ ]:
# %%
# === CONFIG ===
CSV_PATH = "AI/synthetic_dataset (1).csv"   # path to your CSV
WINDOW = 14           
BATCH_SIZE = 64
LR = 1e-3
EPOCHS = 80
PATIENCE = 8         

FEATURE_COLS = ["temp", "humidity", "rainfall", "wqi", "diarrhea", "cholera", "typhoid"]
TARGET_COLS = ["diarrhea", "cholera", "typhoid"]
MODEL_OUT = "cnn_lstm_best.pth"
SEED = 42

# 🔑 Load API key from .env file
load_dotenv()
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
print("OPENWEATHER_API_KEY loaded:", OPENWEATHER_API_KEY is not None)

torch.manual_seed(SEED)
np.random.seed(SEED)


In [ ]:
# %%
df = pd.read_csv(CSV_PATH, parse_dates=["Date"], dayfirst=True, infer_datetime_format=True)
df = df.sort_values("Date").reset_index(drop=True)

df[FEATURE_COLS] = df[FEATURE_COLS].apply(pd.to_numeric, errors='coerce').fillna(0.0)
df[TARGET_COLS] = df[TARGET_COLS].astype(int)

# Add difference features
for col in ["temp", "humidity", "rainfall"]:
    df[f"{col}_diff"] = df[col].diff().fillna(0)

FEATURE_COLS_EXT = FEATURE_COLS + [f"{col}_diff" for col in ["temp", "humidity", "rainfall"]]

print("Dataset preview:")
print(df.head())
print("\nFeature columns:", FEATURE_COLS_EXT)


In [ ]:
# %%
class SeqDataset(Dataset):
    def __init__(self, df, feature_cols, target_cols, window=WINDOW, scaler_x=None, scaler_y=None):
        self.df = df.reset_index(drop=True)
        self.feature_cols = feature_cols
        self.target_cols = target_cols
        self.window = window

        X = self.df[self.feature_cols].values.astype(np.float32)
        Y = self.df[self.target_cols].values.astype(np.float32)

        self.scaler_x = MinMaxScaler() if scaler_x is None else scaler_x
        self.scaler_y = MinMaxScaler() if scaler_y is None else scaler_y
        if scaler_x is None:
            self.scaler_x.fit(X)
        if scaler_y is None:
            self.scaler_y.fit(Y)

        Xs = self.scaler_x.transform(X)
        Ys = self.scaler_y.transform(Y)

        self.samples = []
        for i in range(window, len(self.df)):
            x_win = Xs[i-window:i]
            y_target = Ys[i]
            self.samples.append((x_win.astype(np.float32), y_target.astype(np.float32)))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x, y = self.samples[idx]
        return x, y


In [ ]:
# %%
n = len(df)
train_end = int(0.7 * n)
val_end = int(0.85 * n)

df_train = df.iloc[:train_end].reset_index(drop=True)
df_val = df.iloc[train_end - WINDOW:val_end].reset_index(drop=True)
df_test = df.iloc[val_end - WINDOW:].reset_index(drop=True)

train_ds = SeqDataset(df_train, FEATURE_COLS_EXT, TARGET_COLS, window=WINDOW)
val_ds = SeqDataset(df_val, FEATURE_COLS_EXT, TARGET_COLS, window=WINDOW,
                    scaler_x=train_ds.scaler_x, scaler_y=train_ds.scaler_y)
test_ds = SeqDataset(df_test, FEATURE_COLS_EXT, TARGET_COLS, window=WINDOW,
                     scaler_x=train_ds.scaler_x, scaler_y=train_ds.scaler_y)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

print("Dataset sizes:", len(train_ds), len(val_ds), len(test_ds))


In [ ]:
# %%
class CNNLSTM(nn.Module):
    def __init__(self, n_features, cnn_channels=64, lstm_hidden=128, lstm_layers=1, out_dim=3, dropout=0.3):
        super().__init__()
        self.conv1 = nn.Conv1d(n_features, cnn_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(cnn_channels)

        self.conv2 = nn.Conv1d(cnn_channels, cnn_channels*2, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(cnn_channels*2)

        self.conv3 = nn.Conv1d(cnn_channels*2, cnn_channels*4, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(cnn_channels*4)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

        self.lstm = nn.LSTM(input_size=cnn_channels*4, hidden_size=lstm_hidden,
                            num_layers=lstm_layers, batch_first=True)

        self.fc = nn.Sequential(
            nn.Linear(lstm_hidden, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, out_dim)
        )

    def forward(self, x):
        x = x.permute(0, 2, 1)   # (batch, features, seq)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = x.permute(0, 2, 1)   # (batch, seq, features)
        x, _ = self.lstm(x)
        out = x[:, -1, :]
        return self.fc(out)

model = CNNLSTM(n_features=len(FEATURE_COLS_EXT), out_dim=len(TARGET_COLS)).to(DEVICE)
print(model)


In [ ]:
# %%
def train_model_full_epochs(model, train_loader, val_loader, epochs=EPOCHS, lr=LR):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=4)
    best_val = 1e9

    for epoch in range(1, epochs+1):
        model.train()
        train_losses = []
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            pred = model(X)
            loss = criterion(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        model.eval()
        val_losses = []
        with torch.no_grad():
            for Xv, yv in val_loader:
                Xv, yv = Xv.to(DEVICE), yv.to(DEVICE)
                pv = model(Xv)
                val_losses.append(criterion(pv, yv).item())

        train_loss, val_loss = np.mean(train_losses), np.mean(val_losses)
        scheduler.step(val_loss)
        print(f"Epoch {epoch}: Train {train_loss:.6f}, Val {val_loss:.6f}")

        # Save model if validation improves
        if val_loss < best_val - 1e-6:
            best_val = val_loss
            torch.save(model.state_dict(), MODEL_OUT)
            print(f"  Saved best model at epoch {epoch}")

    model.load_state_dict(torch.load(MODEL_OUT, map_location=DEVICE))
    return model

# Train model for all epochs
model = train_model_full_epochs(model, train_loader, val_loader)


In [ ]:
# %%
def classify_risk(disease, value):
    thresholds = {
        "diarrhea": {"low": 30, "medium": 60},
        "cholera": {"low": 50, "medium": 80},
        "typhoid": {"low": 20, "medium": 40}
    }
    
    if value <= thresholds[disease]["low"]:
        return "LOW ✅ — normal levels"
    elif value <= thresholds[disease]["medium"]:
        return "MEDIUM ⚠️ — monitor closely"
    else:
        return "HIGH 🚨 — take preventive measures"



In [ ]:
# %%
def predict_next_day(model, df_recent, lat, lon):
    """Use live weather + model to predict next-day diseases"""
    try:
        weather_data = requests.get(
            f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={OPENWEATHER_API_KEY}&units=metric"
        ).json()
        if "main" not in weather_data:
            raise ValueError("Weather API returned no 'main'")
        live_weather = {
            'temp': round(weather_data['main']['temp'], 2),
            'humidity': round(weather_data['main']['humidity'], 2),
            'rainfall': round(weather_data.get('rain', {}).get('1h', 0), 2)
        }
        print("Weather API raw response:", weather_data)
    except Exception as e:
        print("⚠️ Weather API failed:", e)
        live_weather = {
            'temp': df_recent['temp'].iloc[-1],
            'humidity': df_recent['humidity'].iloc[-1],
            'rainfall': df_recent['rainfall'].iloc[-1]
        }

    df_next = df_recent.copy()
    df_next['temp_diff'] = df_next['temp'].diff().fillna(0)
    df_next['humidity_diff'] = df_next['humidity'].diff().fillna(0)
    df_next['rainfall_diff'] = df_next['rainfall'].diff().fillna(0)

    df_next.iloc[-1, df_next.columns.get_loc('temp')] = live_weather['temp']
    df_next.iloc[-1, df_next.columns.get_loc('humidity')] = live_weather['humidity']
    df_next.iloc[-1, df_next.columns.get_loc('rainfall')] = live_weather['rainfall']

    df_next.iloc[-1, df_next.columns.get_loc('temp_diff')] = df_next['temp'].iloc[-1] - df_next['temp'].iloc[-2]
    df_next.iloc[-1, df_next.columns.get_loc('humidity_diff')] = df_next['humidity'].iloc[-1] - df_next['humidity'].iloc[-2]
    df_next.iloc[-1, df_next.columns.get_loc('rainfall_diff')] = df_next['rainfall'].iloc[-1] - df_next['rainfall'].iloc[-2]

    X_next = df_next[FEATURE_COLS_EXT].values.astype(np.float32)
    X_next_scaled = train_ds.scaler_x.transform(X_next)
    X_next_tensor = torch.tensor(X_next_scaled).unsqueeze(0).to(DEVICE)

    model.eval()
    with torch.no_grad():
        pred_scaled = model(X_next_tensor).cpu().numpy()

    pred = train_ds.scaler_y.inverse_transform(pred_scaled)[0]
    # Round predictions to integer
    pred_int = [int(round(p)) for p in pred]
    result = {col: pred_int[i] for i, col in enumerate(TARGET_COLS)}
    result.update(live_weather)
    return result


In [ ]:
# %%
recent_df = df.iloc[-WINDOW:].copy()
next_day_pred = predict_next_day(model, recent_df, lat=13.6576 , lon=78.2624)

print("🌍 Next-day Disease Forecast with Risk Alerts")
for disease in ["diarrhea", "cholera", "typhoid"]:
    value = next_day_pred[disease]
    alert = classify_risk(disease, value)
    print(f"{disease.capitalize()}: {value} cases → {alert}")

print(f"\nWeather Conditions: 🌡️ {next_day_pred['temp']}°C, 💧 {next_day_pred['humidity']}%, 🌧️ {next_day_pred['rainfall']}mm")
